1. Loads csv as pandas dataframe
2. Converts each line of the dataframe to tensor
3. Creates _MapDataset obj (same as obj recieved when loading sample data (TensorFlow2))
4. Preprocess data -> needs to convert to array and back to tensor to make operations -> CHANGE!! preprocess the csv before to simplify
5. Embed both towers and train the model
6. Obtain results

In [1]:
import tqdm as notebook_tqdm
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

import pandas as pd
import numpy as np
import tensorflow as tf
import pandas as pd
import functions as f

from typing import Dict, Text
import pprint

/home/pablo/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-11-08 08:51:05.175993: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-08 08:51:05.176073: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-08 08:51:05.183573: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-08 08:51:06.176155: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is op

In [3]:
#we get the customers and article data
# customers_path = '../00 - Data/customers/customers.csv'

#we read dataset, cut it short, write it to csv, and read it again
transactions_path = '../00 - Data/transactions_train/transactions_train.csv'
short_transactions_path = f.shorten_transactions(transactions_path,1)

articles_path = '../00 - Data/articles/articles.csv'

Number of elements in database before : 31788324
Number of elements in database after : 1179208
Database contains recrods from : 2020-09-15 00:00:00 to : 2020-08-15 00:00:00


In [4]:
batch_size  = 64
articles = tf.data.TextLineDataset(articles_path).skip(1)  # Skip the header
articles = articles.map(f.parse_csv_line_a, num_parallel_calls=tf.data.experimental.AUTOTUNE) # creates _ParallelMapDataset
# transactions = transactions.batch(batch_size)  # Choose an appropriate batch size -> creates _BatchDataset

transactions = tf.data.TextLineDataset(short_transactions_path).skip(1)  # Skip the header
transactions = transactions.map(f.parse_csv_line_t, num_parallel_calls=tf.data.experimental.AUTOTUNE) # creates _ParallelMapDataset
# transactions = transactions.batch(batch_size)  # Choose an appropriate batch size -> creates _BatchDataset

In [5]:
# limiting the features for testing
transactions = transactions.map(lambda x:{'customer_id':x['customer_id'],'article_id':x['article_id'],'t_dat':x['t_dat']})
articles = articles.map(lambda x:{'article_id':x['article_id']})

In [6]:
transactions

<_MapDataset element_spec={'customer_id': TensorSpec(shape=(), dtype=tf.string, name=None), 'article_id': TensorSpec(shape=(), dtype=tf.float32, name=None), 't_dat': TensorSpec(shape=(), dtype=tf.string, name=None)}>

In [13]:
customer_id_vocabulary = tf.keras.layers.experimental.preprocessing.StringLookup(mask_token=None)
customer_id_vocabulary.adapt(transactions.map(lambda x: x['customer_id']))

t_dat_vocabulary = tf.keras.layers.experimental.preprocessing.StringLookup(mask_token=None)
t_dat_vocabulary.adapt(transactions.map(lambda x: x['t_dat']))

# Convert the symbolic tensor to a Python list for article_id
article_id_data = [x['article_id'].numpy() for x in articles]

# Adapt the article_id vocabulary
article_id_vocabulary = tf.keras.layers.experimental.preprocessing.StringLookup(mask_token=None)
article_id_vocabulary.adapt(article_id_data)

In [14]:
class hmModel(tfrs.Model):
 
  def __init__(
      self,
      user_model: tf.keras.Model,
      article_model: tf.keras.Model,
      task: tfrs.tasks.Retrieval):
    super().__init__()
 
    # Set up user and movie representations.
    self.user_model = user_model
    self.article_model = article_model
 
    # Set up a retrieval task.
    self.task = task
 
  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # Define how the loss is computed.
 
    user_embeddings = self.user_model(features["customer_id"])
    article_model = self.article_model(features["article_id"])
 
    return self.task(user_embeddings, article_model)


In [16]:
users_model = tf.keras.Sequential([transactions,tf.keras.layers.Embedding(customer_id_vocabulary.vocab_size(),64)])
article_model = tf.keras.Sequential([articles,tf.keras.layers.Embedding(article_id_vocabulary.vocab_size(),64)])
 
task = tfrs.tasks.Retrieval(metrics=tfrs.metrics.FactorizedTopK(article_id_data))

TypeError: The added layer must be an instance of class Layer. Received: layer=<_MapDataset element_spec={'customer_id': TensorSpec(shape=(), dtype=tf.string, name=None), 'article_id': TensorSpec(shape=(), dtype=tf.float32, name=None), 't_dat': TensorSpec(shape=(), dtype=tf.string, name=None)}> of type <class 'tensorflow.python.data.ops.map_op._MapDataset'>.

In [20]:
model = hmModel(users_model,article_model,task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))
model.fit(transactions.batch(4096), epochs=3)

Epoch 1/3


AttributeError: in user code:

    File "/home/pablo/.local/lib/python3.10/site-packages/keras/src/engine/training.py", line 1377, in train_function  *
        return step_function(self, iterator)
    File "/home/pablo/.local/lib/python3.10/site-packages/keras/src/engine/training.py", line 1360, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/pablo/.local/lib/python3.10/site-packages/keras/src/engine/training.py", line 1349, in run_step  **
        outputs = model.train_step(data)
    File "/home/pablo/.local/lib/python3.10/site-packages/tensorflow_recommenders/models/base.py", line 68, in train_step
        loss = self.compute_loss(inputs, training=True)
    File "/tmp/ipykernel_7682/341823061.py", line 23, in compute_loss
        return self.task(user_embeddings, article_model)
    File "/home/pablo/.local/lib/python3.10/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceb

In [ ]:
recommends = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
recommends.index_from_dataset(articles.batch(100).map(lambda title: (title, model.article_model(title))))
 
id_ = input('Enter the user_id: ')
_, titles = recommends(np.array([str(id_)]))
print('Top recommendation for user',id_,titles[0, :3])